In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tqdm import tqdm
import os
from matplotlib import pyplot as plt

### load dataset from directory

In [ ]:
batch_size = 32
IMG_SIZE = (160, 160)

train_dir = 'data/images/binary_classification/training'
test_dir = 'data/images/binary_classification/test'

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=IMG_SIZE,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
train_dir,
validation_split=0.2,
subset="validation",
seed=42,
image_size=IMG_SIZE,
batch_size=batch_size)

#test dataset
# test_ds = tf.keras.utils.image_dataset_from_directory(
# test_dir,
# image_size=IMG_SIZE,
# batch_size=batch_size)

### Configure the dataset for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE #find a good allocation of CPU budget automatically

#use buffered prefetching, to yield data from disk without having I/O become blocking
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Image Preview

In [ ]:
class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

### Rescale pixel values

In [12]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

### Create the base model from the pre-trained ResNet50

In [ ]:
# Create the base model from the pre-trained model ResMet50V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

#### Freeze the convolutional base

In [ ]:
base_model.trainable = False

### Basemodel summary

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Add a classification head

In [ ]:
# converts the features (output of base model) to a single 1280-element vector per image
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

### Add Dense Layer

In [ ]:
#No activation Function needed because positive numbers predict class 1, negative numbers predict class 0
prediction_layer = tf.keras.layers.Dense(1)

### Build a model

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = rescale(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compile the model

In [ ]:
#Compile the model before training it. Since there are two classes,
#use tf.keras.losses.BinaryCrossentropy loss with from_logits=True since the model provides a linear output.
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

### Train the model

In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_ds)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=validation_ds)

### Learning Curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

#Note: If you are wondering why the validation metrics are clearly better than the training metrics,
#the main factor is because layers like tf.keras.layers.BatchNormalization and tf.keras.layers.Dropout
#affect accuracy during training. They are turned off when calculating validation loss.

### Fine tuning

Read before you go on:

This should only be attempted after you have trained the top-level classifier with the pre-trained model set to non-trainable. If you add a randomly initialized classifier on top of a pre-trained model and attempt to train all layers jointly, the magnitude of the gradient updates will be too large (due to the random weights from the classifier) and your pre-trained model will forget what it has learned.